《[利用Python进行数据分析](https://book.douban.com/subject/25779298/)》读书笔记。
 
 [第7章](/2017/03/10/python_data_analysis7.html)  第4节：字符串操作

所有用到的数据可以从[作者的 github](https://github.com/wesm/pydata-book)下载。


In [67]:
%pylab inline
import pandas as pd
from pandas import Series, DataFrame

Populating the interactive namespace from numpy and matplotlib


Python有简单易用的字符串和文本处理功能。大部分文本运算直接做成了字符串对象的内置方法。当然还能用正则表达式。pandas对此进行了加强，能够对数组数据应用字符串表达式和正则表达式，而且能处理烦人的缺失数据。

# 字符串对象方法

对于大部分的字符串而言，内置的方法已经能够满足要求了。

python 的字符串方法主要有：

- count
- endswith, startswith
- join
- index
- find
- rfind
- replace
- strip, rstrip, lstrip
- split
- lower, upper
- ljust, rjust

In [3]:
# 返回一个列表
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

In [4]:
#  去除空格
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [5]:
# + 连接字符串。 注意下面的赋值方式
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [6]:
# 上面的不实用，下面是一种更快的风格
'::'.join(pieces)

'a::b::guido'

In [7]:
# 字串定位，常用的有 in、index、find
'guido' in val

True

In [8]:
val.index(',')

1

In [9]:
val.find(':')

-1

In [11]:
# 不包含子串会报错
# val.index(':') 

In [13]:
# 返回个数
val.count(',')

2

In [14]:
#  替换
val.replace(',', '::')

'a::b::  guido'

In [15]:
#  剔除
val.replace(',', '')

'ab  guido'

# 正则表达式

正则表达式（regex）提供了一种灵活的在文本中搜索、匹配字符串的模式。用的是re模块。

re模块的函数分为3类：模式匹配、替换、拆分。

关于python 内置的正则表达式(re 模块），可以参考[AstralWind的总结](http://www.cnblogs.com/huxi/archive/2010/07/04/1771073.html)。

另外animalize 介绍了 [更强大的第三方模块(regex)](http://www.cnblogs.com/animalize/p/4949219.html)。

re 模块的主要方法有：

- findall, finditer
- match
- search
- split
- sub, subn

In [44]:
import re
text = "foo    bar\t baz  \tqux"

# 先编译正则表达式 \s+ (多个空白字符)，然后再调用split
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [45]:
# 等价于
# 如果想对许多字符串都应用同一条正则表达式，应该先compile节省时间
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [46]:
# 找到匹配regex的所有模式 (\s+)
regex.findall(text)

['    ', '\t ', '  \t']

In [47]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
# r， 指定为原生字符串，使得转义字符 \ 不起作用
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [41]:
#findall 返回字符串中所有匹配项
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [51]:
# search只返回第一个匹配项
# 返回的是一种特殊特殊对象，这个对象只能告诉我们模式在原始字符串中的起始和结束位置
m = regex.search(text)
m

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [49]:
text[m.start():m.end()]

'dave@google.com'

In [53]:
# match更加严格，它只匹配出现在字符串开头的模式
regex.match(text)

In [56]:
# sub方法，会将匹配到的模式替换为指定字符串，并返回新字符串
regex.sub('REDACTED', text)

'Dave REDACTED\nSteve REDACTED\nRob REDACTED\nRyan REDACTED\n'

In [58]:
# 如果想将找出的模式分段， 需要用圆括号括起来
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [60]:
m = regex.match('wesm@bright.net')
m.groups() # 返回 tuple

('wesm', 'bright', 'net')

In [61]:
regex.findall(text) # 返回列表

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [62]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text)) # 返回替换后的字符串

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



In [63]:
regex = re.compile(r"""
    (?P<username>[A-Z0-9._%+-]+)
    @
    (?P<domain>[A-Z0-9.-]+)
    \.
    (?P<suffix>[A-Z]{2,4})""", flags=re.IGNORECASE|re.VERBOSE)

In [65]:
m = regex.match('wesm@bright.net')
m.groupdict()   #  返回一个简单的字典

{'domain': 'bright', 'suffix': 'net', 'username': 'wesm'}

# pandas中矢量化字符串函数

 将字符串方法或正则表达式应用到一系列数据。常用的方法包括：
 
 - cat
 - contains
 - count
 - endswith, startswith
 - findall
 - get
 - join
 - len
 - lower, upper
 - match
 - pad
 - center
 - repeat
 - replace
 - slice
 - split
 - strip, rstrip, lstrip
 
通过data.map()方法，所有字符串和正则都能传入各个值（通过lambda或者其他函数），但是如果存在NA就会报错。 #然而，Series有些跳过NA的方法。通过Series的str属性可以访问这些方法。

In [68]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = Series(data)

In [69]:
data

Dave     dave@google.com
Rob        rob@gmail.com
Steve    steve@gmail.com
Wes                  NaN
dtype: object

In [70]:
data.isnull()

Dave     False
Rob      False
Steve    False
Wes       True
dtype: bool

In [71]:
data.str.contains('gmail')

Dave     False
Rob       True
Steve     True
Wes        NaN
dtype: object

In [72]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [73]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Rob        [(rob, gmail, com)]
Steve    [(steve, gmail, com)]
Wes                        NaN
dtype: object

In [75]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

/Users/holbrook/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: In future versions of pandas, match will change to always return a bool indexer.
  if __name__ == '__main__':


Dave     (dave, google, com)
Rob        (rob, gmail, com)
Steve    (steve, gmail, com)
Wes                      NaN
dtype: object

In [77]:
#有两个办法可以实现矢量化的元素获取操作：要么使用str.get,要么在str属性上用索引
matches.str.get(1)

Dave     google
Rob       gmail
Steve     gmail
Wes         NaN
dtype: object

In [78]:
matches.str[0]

Dave      dave
Rob        rob
Steve    steve
Wes        NaN
dtype: object

In [79]:
# 进行截取
data.str[:5]

Dave     dave@
Rob      rob@g
Steve    steve
Wes        NaN
dtype: object